# Deep Learning
 - CNN

- CNN의 등장 배경
 - CNN은 기존의 Deep Neural Network에 한계를 극복하고자 출현하였다.
 - 기존의 DNN은 Input Layer에 Data를 Vector형태로 Input하는데 이렇게 되면 Data의 Feature수가 많으면 가중치를 Feature의 수만큼 입력해야 하는 문제가 발생했다.
 - 예를들면 28 by 28 Fixel의 Image를 DNN으로 학습시키려면 28 by 28(=784)만큼의 가중치를 설정해 주어야 한다. 
   그럼 W벡터의 각 element를 계산하는데(연산은 실제로 W를 연산하는 것이 핵심) Memory상의 문제가 발생하게 되는것이다.

 - 해결방법으로 나온것이 CNN이며 Input Layer에 일차원의 데이터를 입력하는 것이 아니라 n차원의 데이터를 집적 입력하여 Data Matrix와 가중치를 합성곱한다면 문제를 해결할수 있다고 하였다.
 - 예를 들면 28 by 28 을 3by3 의 가중치 Matrix로 Convolution Operation을 수행한다면 X*W=y의 형태는 유지하되(곱셈의 종류는 바뀌었음) 가중치의 element를 9(3 by 3)만 설정하면 되어 메모리 문제를 해결할 수 있게 된 것이다.
 
 
- X*W + b = y
 - Convolution 연산에 대해서는 따로 다루도록 하자. Linear Algebra필요!!

mnist데이터를 통해 CNN을 구현하면서 구체화

In [0]:
import os, warnings
warnings.filterwarnings(action='ignore')

In [6]:
## 텐서플로와 케라스 파이토치가 딥러닝의 3대장 그 근본인 tensorflow를 이해하면 keras와 pytorch는 쉽게 다룰수 있지 않을까 생각
import tensorflow as tf
## onehot encoding을 적용시키면서 mnist라는 변수에 mmnist데이터를 할당시킨다.
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/',one_hot=True)

## keras.datasets에도 mnist데이터가 있는걸로 알고있다.

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


- tensorflow에는 그래프의 생성과 실행으로 나뉘어져 있어 입력받는 값을 placeholder로 주어 session을 실행시킬때 입력시키는 식의 실행을 한다.
 아직 직관적으로 잘 모르겠다.

In [0]:
X=tf.placeholder(tf.float32,[None,28,28,1])
Y=tf.placeholder(tf.float32,[None,10])

- 위의 placeholder로 []안의 shape에 맞게 입력값을 받을 빈 행렬을 생성하였다. 그러면 shape가 저런 형태가 의미하는 게 뭔지 알아보자.


In [20]:
import numpy as np
## 1차원
arr_1=np.ones(5)
print('shape:',arr_1.shape)
print(arr_1)

## 2차원
arr_2=np.ones([2,5])
print('shape:',arr_2.shape)
print(arr_2)

## 3차원
arr_3=np.ones([2,3,5])
print('shape:',arr_3.shape)
print(arr_3)

## 4차원
arr_4=np.ones([2,3,4,5])
print('shape:',arr_4.shape)
print(arr_4)

shape: (5,)
[1. 1. 1. 1. 1.]
shape: (2, 5)
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
shape: (2, 3, 5)
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]]
shape: (2, 3, 4, 5)
[[[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]]


- 잘 보면 2차원의 arr는 5by (1)이 두개 , 3차원 arr는 3 by 5 matrix가 두개 , 4차원 arr는 (3 by 4 by 5)의 arr가 두개가 있는 형태다. 
- 따라서 n dim arr는 [A,B]라고 했을때 A는 count, B는 arr의 형태라고 보면 된다. (B도 이와같이 나눠갈수 있다.)
- 그럼 X=tf.placeholder([None,32,32,1])이 의미하는 것은 개수는 주어지지 않았으나 32,32,1의 3차원 행렬을 n개 집어넣겠다는 의미, Y는 10by1 벡터를 n개 집어넣겠다는 의미

In [0]:
W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [45]:
W2=tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [0]:
W3=tf.Variable(tf.random_normal([7*7*64,128],stddev=0.01))
L3=tf.reshape(L2,[-1,7*7*64])
L3=tf.matmul(L3,W3) ## FC layer
L3=tf.nn.relu(L3)

In [0]:
W4=tf.Variable(tf.random_normal([128,10],stddev=0.01))
model=tf.matmul(L3,W4)

In [0]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)

In [0]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

In [51]:
batch_size=1000
iter_n=int(mnist.train.num_examples/batch_size) ## total_batch


for epoch in range(5):
    avg_cost=0
    for i in range(iter_n):

        x_batch,y_batch=mnist.train.next_batch(batch_size)
        #        x_batch = X[i*batch_size:(i+1)*batch_size]
        #        y_batch = Y[i*batch_size:(i+1)*batch_size]
        
        x_batch=x_batch.reshape(-1,28,28,1)
        
        _, loss=sess.run([optimizer,cost],
                        feed_dict={X:x_batch,
                                  Y:y_batch})
        
        avg_cost += loss
        
    print('Avg Cost : {:.3f}'.format(avg_cost/iter_n))

Avg Cost : 0.775
Avg Cost : 0.091
Avg Cost : 0.055
Avg Cost : 0.037
Avg Cost : 0.032


In [52]:
is_correct=tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
acc=tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('정확도:',sess.run(acc,
                     feed_dict={X:mnist.test.images.reshape([-1,28,28,1]),Y:mnist.test.labels}))

정확도: 0.9881


- 해석은 다음에하자. 저거 [3,3,1,32]를 1by 32 의 세층짜리, 세개를 표준편차 0.01인 정규분포의 임의의 수로 채워넣어 변수로 할당하는 것이 W1인데 직관적으로 잘 이해가 안된다.
- 직관적인 해석은 입력받은 X를 3 by 3 인 kernel로 convolution하여 32개의 kernel을 만들어 32개의 node L1을 구성하여
- 가중치를 거쳐 활성화함수 L1을 적용하고
- 2 by 2 인 kernel로 step 2씩 pooling하는 것이라고 이해하면 될 것 같은데 그러면 [3,3,1,32]를 구조적으로 설명하기 어려워진다.;; 
- convolution의 구조에 대해 좀 더 공부해야할 필요가 있어보임


In [35]:
arr_5=np.ones([3,3,1,4])
print(arr_5)

[[[[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]]


 [[[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]]


 [[[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]

  [[1. 1. 1. 1.]]]]


In [37]:
arr_6=np.ones([2,5,5,1])
print(arr_6)

[[[[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]]


 [[[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   [1.]
   [1.]]]]


In [31]:
mul_1=np.convolve(arr_6,arr_5)

ValueError: ignored

In [0]:
?tf.nn.conv2d

In [0]:
arr_8=np.array(tf.random_normal([3,3,1,32],stddev=0.01))

In [44]:
tf.Session().run(tf.random_normal([3,3,1,32],stddev=0.01))

array([[[[ 1.55713614e-02,  1.83089878e-02,  6.78105326e-03,
          -1.10584619e-02, -1.22201154e-02,  1.68330949e-02,
          -1.12038637e-02, -1.13467465e-03,  1.35456081e-02,
           9.05543857e-04,  1.71301886e-02, -1.04857888e-03,
           5.04512992e-03, -5.00222668e-03, -8.92461743e-03,
           9.51402634e-03,  1.97741836e-02, -1.05701853e-02,
          -1.69092754e-03, -1.53764738e-02, -4.28797863e-03,
           5.49293449e-03,  8.85391608e-03, -2.83068861e-03,
           2.73332354e-02,  6.27857726e-03,  2.30175164e-02,
          -1.26524903e-02,  6.38614874e-03,  7.12338928e-03,
          -6.06228691e-03,  5.28737670e-03]],

        [[-1.91382486e-02,  1.24295913e-02,  7.23313540e-03,
          -8.91594309e-03, -2.39818986e-03, -2.95247184e-04,
           4.35314560e-03, -4.58858209e-03,  5.19456714e-03,
           4.69664065e-03, -1.11843543e-02,  7.72418920e-03,
          -3.49282613e-03, -1.00117382e-02,  8.52631219e-03,
          -8.41965340e-03,  2.37711193